# Ryhmä-190

## Python-paketit

In [69]:
# Jupyter-notebookissa voi asentaa paketit samaan tapaan kuin komentoriviltä.
# Tähän käytetään "magic commandeja", jotka alkavat %-merkillä.
# Näiden asentamisessa voi mennä muutama minuutti, mutta sen jälkeen niitä ei tarvitse asentaa uudestaan. 
#%pip install seaborn
#%pip install mlflow azureml azureml-core azureml-mlflow azure-identity

In [70]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, make_scorer, r2_score
from sklearn.model_selection import KFold, cross_val_score
#from azureml.core import Workspace
#from azure.identity import InteractiveBrowserCredential
#import mlflow

## MLFlow-seuranta (ei tällä hetkellä käytössä)

Tarkistetaan ajoympäristö (Azure vai oma kone) ja autentikoidutaan sen mukaan.

In [71]:
def is_running_in_azure():
    return 'AZUREML_RUN_ID' in os.environ

def get_workspace():
    if is_running_in_azure():
        return Workspace.from_config()
    else:
        return Workspace(subscription_id='1c0e26b6-0fcb-4b6d-911c-2a0836275ea4',
                         resource_group='rg-AML',
                         workspace_name='aml-data_science_masters')
#ws = get_workspace()

In [72]:
#mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
#experiment_name = 'group-190-tracking'
#mlflow.set_experiment(experiment_name)
#mlflow.autolog()

In [73]:
def evaluate_model(model, X_train, y_train, random_state=42):
    ### Train loss
    y_train_pred = model.predict(X_train)
    train_loss = mean_squared_error(y_train, y_train_pred)

    ### 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    mse_scorer = make_scorer(mean_squared_error)
    cv_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring=mse_scorer)
    cv_loss_mean = cv_scores.mean()

    r2_train = r2_score(y_train, y_train_pred)
    r2_cv = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2').mean()

    print('Train loss:', train_loss)
    print('CV loss mean:', cv_loss_mean)
    print('Train R^2:', r2_train)
    print('CV R^2:', r2_cv)


## Tietoaineistojen lataaminen

In [74]:
df_train = pd.read_csv('../data/train.csv', encoding='utf-8', header=0)
df_test = pd.read_csv('../data/test.csv', encoding='utf-8', header=0)

## Esiprosessointi

In [75]:
df_train['parentspecies'] = df_train['parentspecies'].astype('category')
df_train['parentspecies'] = df_train['parentspecies'].cat.codes
df_test['parentspecies'] = df_test['parentspecies'].astype('category')
df_test['parentspecies'] = df_test['parentspecies'].cat.codes
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26637 entries, 0 to 26636
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID                            26637 non-null  int64  
 1   log_pSat_Pa                   26637 non-null  float64
 2   MW                            26637 non-null  float64
 3   NumOfAtoms                    26637 non-null  int64  
 4   NumOfC                        26637 non-null  int64  
 5   NumOfO                        26637 non-null  int64  
 6   NumOfN                        26637 non-null  int64  
 7   NumHBondDonors                26637 non-null  int64  
 8   NumOfConf                     26637 non-null  float64
 9   NumOfConfUsed                 26637 non-null  float64
 10  parentspecies                 26637 non-null  int8   
 11  C=C (non-aromatic)            26637 non-null  int64  
 12  C=C-C=O in non-aromatic ring  26637 non-null  int64  
 13  h

# Feature engineering

Seuraavassa taustaa. Ajatuksena on luoda "meaningful features". Bayesian Data Analysis kurssilla puhuttiin Generalized Linear Regression yhteydessä, että on kannattavaa olla tarkkana datasetin kanssa. Esimerkissä käytettiin puun lehtien painon arviointia. Siinä yksittäiset mitat kuten puun korkeus leveys yms ei antanut kovin hyvää mallia. Näistä kuitenkin pystyttiin luomaan uusia predictoreita, kuten puun muoto, lehtiosan volyymi jne.

Koska me emme ole alan asiantuntijoita, tulkitsen tekoälyn [käyttösääntöjä](https://studies.helsinki.fi/kurssit/toteutus/hy-opt-cur-2425-b5ccfa1b-ac12-4a9a-bef8-b46c0e808555/DATA11002) kurssilla, että voidaan kysyä mitä näistä voidaan laskea. Joten näin tehtiin ja saatiin seuraavia kaavoja.

BDA kurssilla myös poistettiin tämän jälkeen alkuperäiset arvot sotkemasta!

In [76]:
def transform_gbr(orig_df):
    df = orig_df.copy()
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)

    # Add meaningful features
    df['AtomFraction'] = (df['NumOfC'] + df['NumOfO'] + df['NumOfN']) / df['NumOfAtoms']
    df['Polarity'] = df['NumHBondDonors'] / df['MW']
    df['HBondDensity'] = df['NumHBondDonors'] / df['NumOfAtoms']
    df['GroupDensity_CarboxylicAcid'] = df['carboxylic acid'] / df['MW']
    df['Unsaturation'] = df['C=C (non-aromatic)'] + df['C=C-C=O in non-aromatic ring']
    df['ConfigurationalComplexity'] = df['NumOfConf'] / df['MW']

    # df['C:O_ratio'] = df['NumOfC'] / df['NumOfO']
    # df['C:N_ratio'] = df['NumOfC'] / df['NumOfN']
    # df['Percent_C'] = (df['NumOfC'] / df['NumOfAtoms'])
    # df['Percent_O'] = (df['NumOfO'] / df['NumOfAtoms'])
    # df['Percent_N'] = (df['NumOfN'] / df['NumOfAtoms'])
    # df['HBD_fraction'] = df['NumHBondDonors'] / df['NumOfAtoms']
    # df['FractionOfConfsUsed'] = df['NumOfConfUsed'] / df['NumOfConf']
    # df['Has_Conjugated_System'] = np.where((df['C=C-C=O in non-aromatic ring'] > 0) | (df['C=C (non-aromatic)'] > 0), 1, 0)

    # X_train_gbr['NumOfConf'] = np.log(X_train_gbr['NumOfConf'])
    # X_train_gbr['NumOfConfUsed'] = (X_train_gbr['NumOfConfUsed'] == 40).astype(int)
    drop_cols = [
        # 'NumOfC',
        # 'NumOfO',
        # 'NumOfN',
        # 'NumOfAtoms',
        # 'NumHBondDonors',
        # 'carboxylic acid',
        # 'MW',
        # 'C=C-C=O in non-aromatic ring',
        # 'C=C (non-aromatic)',
        # 'NumOfConf',
        # 'NumOfConfUsed',
        'parentspecies',
    ]
    df = df.drop(columns=drop_cols, axis=1)

    return df

## Mallien kouluttaminen

In [77]:
X_train, y_train = df_train.drop(columns=['log_pSat_Pa', 'ID']), df_train['log_pSat_Pa']
X_test = df_test.drop(columns=['ID'])
X_train_gbr = transform_gbr(X_train)
X_test_gbr = transform_gbr(X_test)
X_train_gbr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26637 entries, 0 to 26636
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   MW                            26637 non-null  float64
 1   NumOfAtoms                    26637 non-null  int64  
 2   NumOfC                        26637 non-null  int64  
 3   NumOfO                        26637 non-null  int64  
 4   NumOfN                        26637 non-null  int64  
 5   NumHBondDonors                26637 non-null  int64  
 6   NumOfConf                     26637 non-null  float64
 7   NumOfConfUsed                 26637 non-null  float64
 8   C=C (non-aromatic)            26637 non-null  int64  
 9   C=C-C=O in non-aromatic ring  26637 non-null  int64  
 10  hydroxyl (alkyl)              26637 non-null  int64  
 11  aldehyde                      26637 non-null  int64  
 12  ketone                        26637 non-null  int64  
 13  c

Erilaisia malleja alla. Kaikki mallit käyttävät samaa nimeä `model`, eli vain viimeisenä koulutettu menee testeihin ja tallentuu.

### 1. Dummy

In [78]:
model = DummyRegressor(strategy='mean')
model.fit(X_train, y_train)

df_test['log_pSat_Pa'] = model.predict(X_test)

### 2. Yksinkertainen regressio

In [79]:
model = LinearRegression()
model.fit(X_train_gbr, y_train)

evaluate_model(model, X_train_gbr, y_train)

Train loss: 2.842328865969468
CV loss mean: 2.849511778366428
Train R^2: 0.708036764384357
CV R^2: 0.707127071706114


### 3. Random Forest

In [ ]:
model = RandomForestRegressor(random_state=190)
model.fit(X_train_gbr, y_train)

evaluate_model(model, X_train_gbr, y_train)

### 4. Gradient Boosting Regressor

In [ ]:
gbr_params = {
    "n_estimators": 100,
    "max_depth": 6,
    "min_samples_split": 10,
    "learning_rate": 0.1,
    "loss": "squared_error",
    "random_state": 42,
    # "subsample": 0.6,
}
    
model = GradientBoostingRegressor(**gbr_params)
model.fit(X_train_gbr, y_train)

evaluate_model(model, X_train_gbr, y_train)



Train loss: 2.0323132250606983
CV loss mean: 2.506041208220943
Train R^2: 0.7912413471652237
CV R^2: 0.7424163443507199


# 5. SVR

In [ ]:
# OPTUNA

import optuna
from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR


def objective(trial):
    gbr_params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 150),
        "max_depth": trial.suggest_int("max_depth", 3, 9),
        "min_samples_split": trial.suggest_int("min_samples_split", 5, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.05, 0.2),
        "loss": "squared_error",
        "random_state": 190,
        "subsample": trial.suggest_float("subsample", 0.3, 0.9),
    }
    # C=1.0, epsilon=0.2, kernel='rbf'
    svr_params = {
        "n_estimators": trial.suggest_int("n_estimators", 120, 140),
        "max_depth": trial.suggest_int("max_depth", 5, 7),
        "min_samples_split": trial.suggest_int("min_samples_split", 8, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.15, 0.25),
        "subsample": trial.suggest_float("subsample", 0.4, 0.5),
        "C": trial.suggest_float("C", 0.5, 1.5),
        
        "epsilon": trial.suggest_float("epsilon", 0.1, 0.5),
        "kernel": 'rbf',
        #"degree": trial.suggest_int("degree", 2, 5),
        #"gamma": trial.suggest_float("gamma", 0.1, 1),
        #"coef0": trial.suggest_float("coef0", 0.1, 1),
        #"shrinking": trial.suggest_categorical("shrinking", [True, False]),
        #"tol": trial.suggest_float("tol", 1e-5, 1e-3),
        #"cache_size": 200,
        #"verbose": False,
        #"max_iter": -1,
    }
        
    model = make_pipeline(StandardScaler(), SVR(**svr_params))

    #model = GradientBoostingRegressor(**gbr_params)

    score = cross_val_score(model, X_train, y_train, cv=5, scoring='r2').mean()
    return score
#evaluate_model(model, X_train_gbr, y_train)

study_name = "group-190-667"
storage = "sqlite:///optuna_190.sqlite3"

study = optuna.create_study(
    direction="maximize",
    #sampler=optuna.samplers.TPESampler(seed=190),
    study_name=study_name,
    storage=storage,
    load_if_exists=True
)

study.optimize(objective, n_trials=5)

loaded_study = optuna.load_study(study_name=study_name, storage=storage)

print(f"The best score: {loaded_study.best_value}")
print(f"The best hyperparameter combination: {loaded_study.best_params}")



[I 2024-11-22 19:31:59,422] A new study created in RDB with name: group-190-667
[I 2024-11-22 19:35:02,314] Trial 0 finished with value: 0.7457164705853575 and parameters: {'n_estimators': 130, 'max_depth': 6, 'min_samples_split': 10, 'learning_rate': 0.17504999486418088, 'subsample': 0.44973238669558613, 'C': 1.1881234857217742, 'epsilon': 0.1623558985519675}. Best is trial 0 with value: 0.7457164705853575.
[I 2024-11-22 19:37:46,744] Trial 1 finished with value: 0.7467980922617024 and parameters: {'n_estimators': 129, 'max_depth': 6, 'min_samples_split': 9, 'learning_rate': 0.18374581703895337, 'subsample': 0.3178657360730103, 'C': 1.4450752607641968, 'epsilon': 0.4399911429992751}. Best is trial 1 with value: 0.7467980922617024.
[I 2024-11-22 19:40:28,281] Trial 2 finished with value: 0.7469854282530618 and parameters: {'n_estimators': 60, 'max_depth': 5, 'min_samples_split': 5, 'learning_rate': 0.06958692126135857, 'subsample': 0.3372638728531775, 'C': 1.4949283574498131, 'epsilon'

The best score: 0.7469854282530618
The best hyperparameter combination: {'n_estimators': 60, 'max_depth': 5, 'min_samples_split': 5, 'learning_rate': 0.06958692126135857, 'subsample': 0.3372638728531775, 'C': 1.4949283574498131, 'epsilon': 0.33644990784856366}


In [ ]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR

model = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2, kernel='rbf',))
model.fit(X_train_gbr, y_train)

evaluate_model(model, X_train_gbr, y_train)

## Ennustuksen tallentaminen

In [ ]:
df_test['log_pSat_Pa'] = model.predict(X_test_gbr.drop(columns=['log_pSat_Pa'], axis=1))
df_test[['ID', 'log_pSat_Pa']].to_csv('../submission/submission.csv', index=False)